In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import linear_model
import torch
from scipy.special import softmax 
from sklearn.preprocessing import StandardScaler

import pyro
import pyro.distributions as dist
from pyro.contrib.autoguide import AutoDiagonalNormal, AutoMultivariateNormal
from pyro.infer import MCMC, NUTS, HMC, SVI, Trace_ELBO
from pyro.optim import Adam, ClippedAdam
from pyro.infer import Predictive
from sklearn.model_selection import train_test_split

from models import model__prior_mu_studentT as model 
from collections import Counter
from pyro.infer import MCMC, NUTS

### Data Processing

In [2]:
scaler = StandardScaler()
df = pd.read_pickle('../pickle/df.pkl')

In [3]:
df_sampled = df.sample(1400,random_state=47)

In [4]:
df_features = df_sampled.iloc[:,:-1].copy()
df_target = df_sampled.iloc[:,-1].copy()

In [5]:
df_features = scaler.fit_transform(df_features)

In [6]:
df_train_features, df_test_features,df_train_target, df_test_target =  train_test_split(df_features,df_target,stratify=df_target,random_state=47, test_size=1/7)

df_train_features, df_val_features,df_train_target, df_val_target =  train_test_split(df_train_features,df_train_target,stratify=df_train_target,random_state=47, test_size=1/6)

In [7]:
df_train_target = df_train_target.to_numpy()
df_test_target = df_test_target.to_numpy()
df_val_target = df_val_target.to_numpy()

In [8]:
D = df_train_features.shape[1]
N_train = df_train_features.shape[0]
N_test = df_test_features.shape[0]
N_val = df_val_features.shape[0]
n_cat = 11 
degF=4
tau=1

### Ancestral Sampling 

In [9]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [24]:
# sample coefficients (beta)
beta = np.random.normal(0,1,size=D)
print("beta:", beta)

beta_array = np.zeros((n_cat,D))

for i in range(n_cat):
    
    beta_array[i,:] = np.random.normal(0,1,size=D)
    
# sample observations (y's)
y = np.zeros((N_train,n_cat))
for n in range(N_train):
    
    probs = np.zeros(n_cat)
    for i in range(n_cat):
        probs[i] = np.array([(np.dot(beta_array[i,:], df_train_features[n,:]))])
        
    p =  softmax(probs)
    y[n,:] = np.random.multinomial(1, p)  #binomial with one trial is equivalent to bernoulli

beta: [-2.13724105  1.37190362  0.88932995 -0.0655204   0.63904618 -0.04616769
 -0.78128545 -1.2183219   1.15312354 -1.15888934 -0.45003776 -0.95070199
 -0.03320277  0.03419961]


In [11]:
label_array = np.array([]) 
for i in y:
    label_array = np.append(label_array,np.argmax(i))

In [12]:
counts = Counter(label_array)

In [13]:
print("Accuracy:", 1.0*np.sum(label_array == df_train_target.flatten()) / len(df_train_target))

Accuracy: 0.088


In [14]:
X_train = torch.tensor(df_train_features).float()
y_train = torch.tensor(df_train_target.flatten()).float()

#### Model training 

In [15]:
degF=4
tau=1

In [17]:
# Run inference in Pyro
nuts_kernel = NUTS(model)
mcmc = MCMC(nuts_kernel, num_samples=2000, warmup_steps=500, num_chains=1)
mcmc.run(X_train, n_cat, degF, tau, y_train)

# Show summary of inference results
mcmc.summary()


Sample: 100%|███████████████████████████████████████| 2500/2500 [20:41,  2.01it/s, step size=3.78e-02, acc. prob=0.822]



                    mean       std    median      5.0%     95.0%     n_eff     r_hat
    alpha[0,0]     -3.90      2.01     -3.91     -7.00     -0.58    366.24      1.00
    alpha[0,1]      3.28      1.52      3.26      0.74      5.65    211.96      1.00
    alpha[0,2]      2.43      1.53      2.39     -0.18      4.78    208.65      1.00
    alpha[0,3]     -4.60      2.04     -4.51     -8.20     -1.56    547.59      1.00
    alpha[0,4]     -2.09      1.97     -2.05     -5.09      1.39    219.85      1.00
    alpha[0,5]      1.79      1.54      1.77     -0.60      4.40    211.09      1.00
    alpha[0,6]      4.01      1.51      3.99      1.46      6.33    213.32      1.00
    alpha[0,7]     -9.42      3.04     -9.23    -14.04     -4.40    272.40      1.00
    alpha[0,8]      1.52      1.55      1.48     -1.21      3.85    223.83      1.00
    alpha[0,9]      3.59      1.51      3.59      1.07      5.91    220.13      1.00
   alpha[0,10]      4.32      1.52      4.29      1.70      6.58

In [23]:
posterior_samples = mcmc.get_samples()

samples_alpha = posterior_samples['alpha'].detach().squeeze()
samples_beta = posterior_samples['beta'].detach().squeeze()
    
    

mean_betas = samples_beta.mean(axis=0)
mean_betas = mean_betas.T
mean_alpha = samples_alpha.mean(axis=0)
mean_alpha = mean_alpha.T

    
y_val_pred = np.zeros((N_val,n_cat))

for n in range(N_val):

    probs = np.zeros(n_cat)
    
    for i in range(n_cat):
        probs[i] = np.array([mean_alpha[i]+(np.dot(mean_betas[i,:], df_val_features[n,:]))])

    p =  softmax(probs)
    y_val_pred[n,:] = np.argmax(p)  #binomial with one trial is equivalent to bernoulli

y_val_pred = y_val_pred[:,0]


print("Accuracy:", 1.0*np.sum(y_val_pred == df_val_target.flatten()) / len(y_val_pred))

Accuracy: 0.47
